# `numba`

Numba je balíček který umožňuje kompilaci Python kódu pomocí [LLVM](http://llvm.org/). 

Podporuje just-in-time kompilaci pomocí dekorátoru `jit` (http://numba.pydata.org/numba-doc/latest/user/jit.html). - tedy kompilaci v okamžiku prvního spuštění.

Podporuje také tzv. ahead-of-time kompilaci pomocí modulu `numba.pycc` (http://numba.pydata.org/numba-doc/latest/user/pycc.html).

In [ ]:
#!pip install numba

## Numba.jit
```
@numba.jit(
    signature=None, 
    nopython=False, 
    nogil=False, 
    cache=False, 
    forceobj=False, 
    parallel=False, 
    error_model='python', 
    fastmath=False, locals={}
)
```
Numba.jit se dá použít jako dekorátor (nebo jako funkce, stejně jako ostatní dekorátory).

Její parametry jsou:
- `signature` - specifikuje typy vstupních a výstupních parametrů. Pokud není specifikováno, použije se automatická detekce typů.
- `nopython` - pokud je `True`, Numba se pokusí vygenerovat kód, který nepoužívá Python C API. Pokud se to nepodaří, vyhodí chybu. Je to jen přepínač, ať vidíme chybu místo fallbacku do objektového módu.
- `nogil` - pokud je `True`, Numba se pokusí vygenerovat kód, který nepoužívá Python GIL. Více k GILu příště.
- `cache` - pokud je `True`, Numba uloží zkompilovaný kód do souboru a příště ho použije. Jinak pro každý kernel vygeneruje nový kód.
- `forceobj` - pokud je `True`, Numba vždy použije objektový mód.
- `parallel` - pokud je `True`, Numba se pokusí paralelizovat cykly.
- `error_model` - specifikuje jak Numba zachází s chybami. Možnosti jsou `python` (vyhodí Python výjimku), `numpy` (vyhodí Numpy výjimku) a `omit` (ignoruje chyby).
- `fastmath` - pokud je `True`, Numba použije rychlejší, ale méně přesnou matematiku. (typicky float místo double precision)
- `locals` - slovník, který specifikuje typy lokálních proměnných. Používá se pro optimalizaci.

In [ ]:
def my_dot_python(a, b):
    result = 0
    for i in range(len(a)):
        result += a[i] * b[i]
    return result

In [ ]:
import numpy as np
a = np.random.rand(1000000)
b = np.random.rand(1000000)

In [ ]:
%time c = my_dot_python(a, b)

In [ ]:
import numba
my_dot_numba = numba.jit(my_dot_python)

In [ ]:
%time c = my_dot_numba(a, b)

In [ ]:
@numba.jit(signature_or_function='float64(float64[:], float64[:])',
           nopython=True,
           fastmath=True,
           locals={'result': numba.float64})
def my_dot_numba2(a, b):
    result = 0
    for i in range(len(a)):
        result += a[i] * b[i]
    return result

In [ ]:
%time c = my_dot_numba2(a, b)

## Numba.stencil
Poměrně pěkný nástroj uvnitř Numby je tzv. stencil kernel. Umožňuje jednoduše a hlavně efektivně implementovat stencil výpočty pomocí relativních indexů.

In [ ]:
from numba import stencil

@stencil()
def kernel1(a):
    tmp_sum = 0.0
    return 0.25 * (a[0, 1] + a[1, 0] + a[0, -1] + a[-1, 0])


In [ ]:
import numpy as np
n = 5
input_arr = np.arange(n*n).reshape((n, n))
# pad with zeros
input_arr = np.pad(input_arr, 1, mode='constant', constant_values=0)
print(input_arr)

In [ ]:
kernel1(input_arr)

Stencil v základní variantě "běhá" v Pythonu, ale dá se jednoduše zakompilovat pomocí Numba.jit.

In [ ]:
from numba import jit

@jit
def kernel2(input_arr):
    @stencil
    def kernel1(a):
        tmp_sum = 0.0
        return 0.25 * (a[0, 1] + a[1, 0] + a[0, -1] + a[-1, 0])
    return kernel1(input_arr)


In [ ]:
kernel2(input_arr)